In [228]:
import pandas as pd

In [229]:
def get_domains(df):
    domains=[]
    for col in df:
      domains.append(list(df[col].unique()))
    return domains

In [230]:
def more_general(h1, h2):
    more_general_parts = []
    for x, y in zip(h1, h2):
        mg = x == "?" or (x != "0" and (x == y or y == "0"))
        more_general_parts.append(mg)
    return all(more_general_parts)

In [231]:
def min_generalizations(h, x):
    h_new = list(h)
    for i in range(len(h)):
        if not more_general( h[i:i+1],x[i:i+1]):
            h_new[i] = '?' if h[i] != '0' else x[i]
    return [tuple(h_new)]

In [232]:
def min_specializations(h, domains, x):
    results = []
    for i in range(len(h)):
        if h[i] == "?":
            for val in domains[i]:
                if x[i] != val:
                    h_new = h[:i] + (val,) + h[i+1:]
                    results.append(h_new)
        elif h[i] != "0":
            h_new = h[:i] + ('0',) + h[i+1:]
            results.append(h_new)
    return results

In [233]:
def generalize_S(x, G, S):
    S_prev = list(S)
    for s in S_prev:
        if s not in S:
            continue
        if not more_general(s, x):
            S.remove(s)
            Splus = min_generalizations(s, x)
            ## keep only generalizations that have a counterpart in G
            S.update([h for h in Splus if any([more_general(g,h) 
                                               for g in G])])
            ## remove hypotheses less specific than any other in S
            S.difference_update([h for h in S if 
                                 any([more_general(h, h1) 
                                      for h1 in S if h != h1])])
    return S

In [234]:
def specialize_G(x, domains, G, S):
    G_prev = list(G)
    for g in G_prev:
        if g not in G:
            continue
        if more_general(g, x):
            G.remove(g)
            Gminus = min_specializations(g, domains, x)
            ## keep only specializations that have a conuterpart in S
            G.update([h for h in Gminus if any([more_general(h, s)
                                                for s in S])])
            ## remove hypotheses less general than any other in G
            G.difference_update([h for h in G if 
                                 any([more_general(g1, h) 
                                      for g1 in G if h != g1])])
    return G

In [235]:
def candidate_elimination(df):
    domains = get_domains(df)[:-1]
    G = set([("?",)*len(domains)])
    S = set([('0',)*len(domains)])
    print(f"\n G[0]:{G}")
    print(f"\n S[0]:{S}")
    for index, row in df.iterrows():
        attribute,target=list(row[:-1]),row[-1] # Splitting data into attributes and decisions
        if target=='Yes': # x is positive example
            G = {g for g in G if more_general(g, attribute)}
            S = generalize_S(attribute, G, S)
        else: # x is negative example
            S = {s for s in S if not more_general(s, attribute)}
            G = specialize_G(attribute, domains, G, S)
        print("-"*100)
        print(f"\n G[{index+1}]:{G}")
        print(f"\n S[{index+1}]:{S}")
    return 

In [236]:
df=pd.read_excel('data.xlsx', sheet_name='Library')
df

,Citations,Size,InLibrary,Price,Editions,Class
0,Some,Small,No,Affordable,One,No
1,Many,Big,No,Expensive,Many,Yes
2,Many,Medium,No,Expensive,Few,Yes
3,Many,Small,No,Affordable,Many,Yes


In [237]:
get_domains(df)

[['Some', 'Many'],
 ['Small', 'Big', 'Medium'],
 ['No'],
 ['Affordable', 'Expensive'],
 ['One', 'Many', 'Few'],
 ['No', 'Yes']]

In [238]:
candidate_elimination(df)


 G[0]:{('?', '?', '?', '?', '?')}

 S[0]:{('0', '0', '0', '0', '0')}
----------------------------------------------------------------------------------------------------

 G[1]:{('?', 'Big', '?', '?', '?'), ('Many', '?', '?', '?', '?'), ('?', 'Medium', '?', '?', '?'), ('?', '?', '?', '?', 'Few'), ('?', '?', '?', 'Expensive', '?'), ('?', '?', '?', '?', 'Many')}

 S[1]:{('0', '0', '0', '0', '0')}
----------------------------------------------------------------------------------------------------

 G[2]:{('?', '?', '?', '?', 'Many'), ('?', 'Big', '?', '?', '?'), ('?', '?', '?', 'Expensive', '?'), ('Many', '?', '?', '?', '?')}

 S[2]:{('Many', 'Big', 'No', 'Expensive', 'Many')}
----------------------------------------------------------------------------------------------------

 G[3]:{('?', '?', '?', 'Expensive', '?'), ('Many', '?', '?', '?', '?')}

 S[3]:{('Many', '?', 'No', 'Expensive', '?')}
-----------------------------------------------------------------------------------------------

In [239]:
df=pd.read_excel('data.xlsx', sheet_name='Disease')
df

,Example,Shape,Size,Color,Surface,Thickness,Class
0,1,Circular,Large,Light,Smooth,Thick,Yes
1,2,Circular,Large,Light,Irregular,Thick,Yes
2,3,Oval,Large,Dark,Smooth,Thin,No
3,4,Oval,Large,Light,Irregular,Thick,Yes


In [240]:
get_domains(df)

[[1, 2, 3, 4],
 ['Circular', 'Oval'],
 ['Large'],
 ['Light', 'Dark'],
 ['Smooth', 'Irregular'],
 ['Thick', 'Thin'],
 ['Yes', 'No']]

In [241]:
candidate_elimination(df)


 G[0]:{('?', '?', '?', '?', '?', '?')}

 S[0]:{('0', '0', '0', '0', '0', '0')}
----------------------------------------------------------------------------------------------------

 G[1]:{('?', '?', '?', '?', '?', '?')}

 S[1]:{(1, 'Circular', 'Large', 'Light', 'Smooth', 'Thick')}
----------------------------------------------------------------------------------------------------

 G[2]:{('?', '?', '?', '?', '?', '?')}

 S[2]:{('?', 'Circular', 'Large', 'Light', '?', 'Thick')}
----------------------------------------------------------------------------------------------------

 G[3]:{('?', '?', '?', 'Light', '?', '?'), ('?', 'Circular', '?', '?', '?', '?'), ('?', '?', '?', '?', '?', 'Thick')}

 S[3]:{('?', 'Circular', 'Large', 'Light', '?', 'Thick')}
----------------------------------------------------------------------------------------------------

 G[4]:{('?', '?', '?', 'Light', '?', '?'), ('?', '?', '?', '?', '?', 'Thick')}

 S[4]:{('?', '?', 'Large', 'Light', '?', 'Thick')}


In [242]:
df=pd.read_excel('data.xlsx', sheet_name='Type')
df

,Size,Color,Shape,Class
0,Big,Red,Circle,No
1,Small,Red,Triangle,No
2,Small,Red,Circle,Yes
3,Big,Blue,Circle,No
4,Small,Blue,Circle,Yes


In [243]:
get_domains(df)

[['Big', 'Small'], ['Red', 'Blue'], ['Circle', 'Triangle'], ['No', 'Yes']]

In [244]:
candidate_elimination(df)


 G[0]:{('?', '?', '?')}

 S[0]:{('0', '0', '0')}
----------------------------------------------------------------------------------------------------

 G[1]:{('?', '?', 'Triangle'), ('?', 'Blue', '?'), ('Small', '?', '?')}

 S[1]:{('0', '0', '0')}
----------------------------------------------------------------------------------------------------

 G[2]:{('Small', '?', 'Circle'), ('Big', '?', 'Triangle'), ('?', 'Blue', '?')}

 S[2]:{('0', '0', '0')}
----------------------------------------------------------------------------------------------------

 G[3]:{('Small', '?', 'Circle')}

 S[3]:{('Small', 'Red', 'Circle')}
----------------------------------------------------------------------------------------------------

 G[4]:{('Small', '?', 'Circle')}

 S[4]:{('Small', 'Red', 'Circle')}
----------------------------------------------------------------------------------------------------

 G[5]:{('Small', '?', 'Circle')}

 S[5]:{('Small', '?', 'Circle')}


In [245]:
df=pd.read_excel('data.xlsx', sheet_name='Face')
df

,Eyes,Nose,Head,Fcolor,Hair,Class
0,Round,Triangle,Round,Purple,Yes,Yes
1,Square,Square,Square,Green,Yes,No
2,Square,Triangle,Round,Yellow,Yes,Yes
3,Round,Triangle,Round,Green,No,No
4,Square,Square,Round,Yellow,Yes,Yes


In [246]:
get_domains(df)

[['Round', 'Square'],
 ['Triangle', 'Square'],
 ['Round', 'Square'],
 ['Purple', 'Green', 'Yellow'],
 ['Yes', 'No'],
 ['Yes', 'No']]

In [247]:
candidate_elimination(df)


 G[0]:{('?', '?', '?', '?', '?')}

 S[0]:{('0', '0', '0', '0', '0')}
----------------------------------------------------------------------------------------------------

 G[1]:{('?', '?', '?', '?', '?')}

 S[1]:{('Round', 'Triangle', 'Round', 'Purple', 'Yes')}
----------------------------------------------------------------------------------------------------

 G[2]:{('?', '?', '?', 'Purple', '?'), ('Round', '?', '?', '?', '?'), ('?', '?', 'Round', '?', '?'), ('?', 'Triangle', '?', '?', '?')}

 S[2]:{('Round', 'Triangle', 'Round', 'Purple', 'Yes')}
----------------------------------------------------------------------------------------------------

 G[3]:{('?', '?', 'Round', '?', '?'), ('?', 'Triangle', '?', '?', '?')}

 S[3]:{('?', 'Triangle', 'Round', '?', 'Yes')}
----------------------------------------------------------------------------------------------------

 G[4]:{('?', '?', 'Round', '?', 'Yes'), ('?', 'Triangle', '?', '?', 'Yes')}

 S[4]:{('?', 'Triangle', 'Round', '?', 'Ye

In [248]:
df=pd.read_excel('data.xlsx', sheet_name='Cars')
df

,Origin,Manufacturer,Color,Decade,Type,Class
0,Japan,Honda,Blue,1980,Economy,Yes
1,Japan,Toyota,Green,1970,Sports,No
2,Japan,Toyota,Blue,1990,Economy,Yes
3,USA,Chrysler,Red,1980,Economy,No
4,Japan,Honda,White,1980,Economy,Yes
5,Japan,Toyota,Green,1980,Economy,Yes
6,Japan,Honda,Red,1990,Economy,No


In [249]:
get_domains(df)

[['Japan', 'USA'],
 ['Honda', 'Toyota', 'Chrysler'],
 ['Blue', 'Green', 'Red', 'White'],
 [1980, 1970, 1990],
 ['Economy', 'Sports'],
 ['Yes', 'No']]

In [250]:
candidate_elimination(df)


 G[0]:{('?', '?', '?', '?', '?')}

 S[0]:{('0', '0', '0', '0', '0')}
----------------------------------------------------------------------------------------------------

 G[1]:{('?', '?', '?', '?', '?')}

 S[1]:{('Japan', 'Honda', 'Blue', 1980, 'Economy')}
----------------------------------------------------------------------------------------------------

 G[2]:{('?', '?', 'Blue', '?', '?'), ('?', '?', '?', 1980, '?'), ('?', 'Honda', '?', '?', '?'), ('?', '?', '?', '?', 'Economy')}

 S[2]:{('Japan', 'Honda', 'Blue', 1980, 'Economy')}
----------------------------------------------------------------------------------------------------

 G[3]:{('?', '?', '?', '?', 'Economy'), ('?', '?', 'Blue', '?', '?')}

 S[3]:{('Japan', '?', 'Blue', '?', 'Economy')}
----------------------------------------------------------------------------------------------------

 G[4]:{('Japan', '?', '?', '?', 'Economy'), ('?', '?', 'Blue', '?', '?')}

 S[4]:{('Japan', '?', 'Blue', '?', 'Economy')}
-------------

In [251]:
df=pd.read_excel('data.xlsx', sheet_name='Day')
df

,sky,airtemp,humidity,wind,water,forcast,Class
0,sunny,warm,normal,strong,warm,same,Yes
1,sunny,warm,high,strong,warm,same,Yes
2,rainy,cold,high,strong,warm,change,No
3,sunny,warm,high,strong,cool,change,Yes


In [252]:
get_domains(df)

[['sunny', 'rainy'],
 ['warm', 'cold'],
 ['normal', 'high'],
 ['strong'],
 ['warm', 'cool'],
 ['same', 'change'],
 ['Yes', 'No']]

In [253]:
candidate_elimination(df)


 G[0]:{('?', '?', '?', '?', '?', '?')}

 S[0]:{('0', '0', '0', '0', '0', '0')}
----------------------------------------------------------------------------------------------------

 G[1]:{('?', '?', '?', '?', '?', '?')}

 S[1]:{('sunny', 'warm', 'normal', 'strong', 'warm', 'same')}
----------------------------------------------------------------------------------------------------

 G[2]:{('?', '?', '?', '?', '?', '?')}

 S[2]:{('sunny', 'warm', '?', 'strong', 'warm', 'same')}
----------------------------------------------------------------------------------------------------

 G[3]:{('sunny', '?', '?', '?', '?', '?'), ('?', 'warm', '?', '?', '?', '?'), ('?', '?', '?', '?', '?', 'same')}

 S[3]:{('sunny', 'warm', '?', 'strong', 'warm', 'same')}
----------------------------------------------------------------------------------------------------

 G[4]:{('sunny', '?', '?', '?', '?', '?'), ('?', 'warm', '?', '?', '?', '?')}

 S[4]:{('sunny', 'warm', '?', 'strong', '?', '?')}
